# 1. Формулировка задачи

| Характеристика | Описание |
| --- | --- |
| **Суть задачи** | Агент получает матрицу смежности взвешенного графа, начальную и конечную вершины. Ему нужно найти кратчайший путь. |
| **Регулировка сложности** | Параметр `difficulty` (от 1 до 10) маппится на гиперпараметры графа: количество вершин (например, `N = difficulty * 2 + 3`) и плотность ребер. |
| **Верификация (Verifier)** | Классический алгоритм Дейкстры рассчитывает эталонный кратчайший путь и его стоимость. Ответ агента парсится, суммируются веса ребер предложенного маршрута, и проверяется, совпадает ли эта сумма с минимальной, а также существует ли такой путь в принципе. |

# 2. Импорты

In [4]:
%cd ..

/


In [2]:
ls

sample_data/


In [6]:

from envs import PathEnv, PathVerifier

ModuleNotFoundError: No module named 'envs'

In [5]:
import torch
print("GPU доступен:" if torch.cuda.is_available() else "GPU недоступен")

GPU доступен:
